In [11]:
from fastapi import FastAPI
from pydantic import BaseModel
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import uvicorn
import nest_asyncio  # 关键：强制兼容事件循环

In [12]:
# 核心操作：让所有异步代码兼容 Colab 已有的事件循环
nest_asyncio.apply()

In [4]:
# 初始化 FastAPI 应用
app = FastAPI(
    title="BERT-MRPC 语义匹配 API",
    description="输入两个英文句子，返回是否语义相似的预测结果（基于 MRPC 数据集训练）",
    version="1.0.0"
)

In [5]:
# 定义 API 请求格式
class SemanticMatchRequest(BaseModel):
    text1: str  # 第一个英文句子（必填）
    text2: str  # 第二个英文句子（必填）

In [6]:
# 加载训练好的 BERT-MRPC 模型和分词器

MODEL_CHECKPOINT_PATH = "/content/drive/MyDrive/bert-mrpc-results/checkpoint-690"

# 加载分词器（和训练时一致，从 Checkpoint 路径加载）
tokenizer = BertTokenizer.from_pretrained(MODEL_CHECKPOINT_PATH)

# 加载模型（自动加载训练好的权重）
model = BertForSequenceClassification.from_pretrained(MODEL_CHECKPOINT_PATH)

# 切换为推理模式（关闭训练时的 Dropout、BatchNorm 等）
model.eval()

# 自动使用 GPU（Colab 已配置 GPU，加速推理）
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"模型加载完成！当前使用设备：{device}")

模型加载完成！当前使用设备：cpu


In [ ]:
# 核心 API 接口（同步函数，无异步冲突）
@app.post("/predict_semantic_match")
def predict_semantic_match(request: SemanticMatchRequest):
    # 分词编码
    encoded_inputs = tokenizer(
        request.text1, request.text2,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128
    ).to(device)

    # 推理
    with torch.no_grad():
        outputs = model(**encoded_inputs)
        probabilities = torch.softmax(outputs.logits, dim=1)
        pred_label = torch.argmax(probabilities, dim=1).item()
        pred_confidence = probabilities[0][pred_label].item()

    # 返回结果
    return {
        "status": "success",
        "input": {"text1": request.text1, "text2": request.text2},
        "prediction": {
            "label": pred_label,
            "label_desc": "语义相似" if pred_label == 1 else "语义不相似",
            "confidence": round(pred_confidence, 4),
            "probabilities": {
                "不相似": round(probabilities[0][0].item(), 4),
                "相似": round(probabilities[0][1].item(), 4)
            }
        }
    }

In [ ]:
# 启动 API 服务（Colab 兼容模式）
if __name__ == "__main__":
    # 关键配置：使用 syncio 循环 + 单工作进程，彻底避免异步冲突
    uvicorn.run(
        app=app,
        host="0.0.0.0",
        port=8000,
        log_level="info",
        workers=1,
        loop="syncio"  # 强制使用同步循环，完全绕开 asyncio 冲突
    )
